# <center>Algorithmes pour le traitement de séquences.<br/>Alignement optimal et logiciel d’aide à la dététection de plagiat</center>
<center>https://perso.esiee.fr/~coustyj/Sequences/IT-4301E-TDm2.html</center>

Importons les paquets utiles pour la suite et chargeons le contenu des fichiers.

In [5]:
import numpy as np
t1 = open('texte1.txt').read()
t2 = open('texte2.txt').read()

## Question 1
**Proposer un algorithme en O(|x| × |y|) pour calculer le score d’un alignement optimal entre x et y (indications : on essaiera de se ramener à un problème connu).**

Pour calculer le score d'un alignement optimal entre x et y, il nous faut dans un premier temps comprendre de quoi il s'agit. En nous référant à la **Définition 3**, on peut lire : "le score d’un alignement (x′,y′) est la grandeur d(x′,y′) égale au nombre de caractères mal appariés entre x′ et y′. Un alignement (x′,y′) de deux séquences x et y définies sur A est dit optimal (pour d) si son score est inférieur ou égal au score de tout autre alignement de x et y."  

Les termes "mal appariés" nous mettent la puce à l'oreille. En effet, dans notre cours *"Comparaison de séquences : distance d'édition"*, nous avions noté : "\[les distances d'édition\] sont définies à partir d'opérations d'édition qui permettent de transformer un mot x en un mot y". Ces opérations (substitution, suppression, insertion) s'appliquent caractère par caractère. Si on associe un coût à chacune de ces opérations (toujours 1 pour la suppression et l'insertion et 1 pour la substitution si les caractères comparés sont différents, 0 sinon), alors faire la somme des coûts des étapes d'une transformation d'un texte 1 en un texte 2 nous permet de compter les "mauvais appariement" et donc revient à calculer le score d'un alignement entre x et y. Il en découle que scorer la transformation d'un texte 1 en un texte 2 comportant les étapes les moins coûteuses, autrement dit la distance d'édition, revient à scorer l'alignement optimal entre x et y.  

Il se trouve que nous connaissons un algorithme qui nous permet de calculer cette distance d'édition : l'algorithme de Levenshtein. Ce dernier nous permet de construire une matrice de dimension (|x|+1)\*(|y|+1), qui pour chaque (i,j) associe la distance d'édition entre xi et yj. Le dernier élément de cette matrice sera donc le score d'un alignement optimal (et de la distance d'édition) entre x et y.

## Question 2

**On considère la matrice T de taille (|x|+1) × (|y|+1) telle que T[i][j] est le score d’un alignement optimal entre xi et yj, où xi et yj désignent les préfixes de x et de y de longueur i et j respectivement.**

**Étant donnée la matrice T (obtenue par exemple avec l’algorithme de la question 1) et les séquences x et y, proposez un schéma de programme retournant un alignement optimal (x′,y′) de x et y.**

**Évaluer la complexité de calcul de l’algorithme proposé; si celle-ci n’est pas linéaire, proposez un autre algorithme de complexité linéaire.**

# Question 3

Implémentez les algorithmes des questions 1 et 2 et testez les en affichant un alignement optimal de texte1.c et texte2.c

In [3]:
def levenshtein_distance(a :str, b:str) -> int:
    n,m = len(a),len(b)
    T = np.zeros((n+1,m+1))
    T[:,0] = range(n+1)
    T[0,:] = range(m+1)
    
    for i in range(n):
        for j in range(m):
            same = int(a[i]!=b[j])
            T[i+1,j+1] = min(
                        T[i, j] + same, # NOTE: Substitution
                        T[i, j+1] + 1, # NOTE: Insertion
                        T[i+1, j] + 1  # NOTE: Delete
                    )
            
    return T

In [11]:
def reverse_lev(T, i, j, x, y):
    
    if i==0:
        x = ['_']*j + x
        return '%s\n%s'%(''.join(x), ''.join(y))
    if j==0:
        y = ['_']*i + y
        return '%s\n%s'%(''.join(x), ''.join(y))
    
    xi, yj = x[i-1], y[j-1]

    if xi == yj:
        return reverse_lev(T, i-1, j-1, x, y)
    
    else:
        
        if T[i-1, j-1] < T[i, j]:
            return reverse_lev(T, i-1, j-1, x, y)

        elif T[i-1, j] < T[i, j]:
            y.insert(j,'_')
            return reverse_lev(T, i-1, j, x, y)
        
        else:
            x.insert(i,'_')
            return reverse_lev(T, i, j-1, x, y)

In [3]:
res = levenshtein_distance(t1,t2)

In [6]:
levenshtein_distance(open('texte1.txt').read(),open('texte2.txt').read())[:,-1]

array([1801., 1800., 1799., ...,  579.,  578.,  577.])

In [7]:
levenshtein_distance('a','b')

array([[0., 1.],
       [1., 1.]])

In [47]:
tt= []
for a,b in zip(t1.split('\n'),t2.split('\n')):
    lev = levenshtein_distance(a,b)[-1,-1]
#     print(lev)
    tt.append(lev)
print(sum(tt))

789.0


In [17]:
T1 = "j'ai écris un texte de ouf"
T2 = "kk kk k k kkkk  kk k kk moi aussi j'ai écris un texte de ouf"
T1 = t1
T2 = t2
T = levenshtein_distance(T1,T2)
print("Distance d'édition : ", T[-1,-1], "\n")
print("Affichage de l'alignement optimal entre T1 et T2\n")
print(reverse_lev(T,T.shape[0]-1,T.shape[1]-1,list(T1),list(T2)))

Distance d'édition :  577.0 

Affichage de l'alignement optimal entre T1 et T2

Source Wik____________i__p_________________________________________________________e_d____i_____a____. La distance de _Levenshtein une distanc______e__ mathematique donnant une mesure de la similarite entre deux chaines de caracteres. Elle est egale au nombre minimal de caracteres qu il faut supprimer, inserer ou _remplacer pour passer d une _chain_e a l autre. Elle a ete proposee par Vladimir Levenshtein en 1965. Elle est______ egalement connue sous les noms de distance d_ ___e_dition ou____________ de deformation dynamique temporelle, notamment en reconnaissance de formes et particulierement en reconnaissance vocale1,2._Cette distance est __d _autant plus grande que le nombre de differences entre les deux chaines est grand. La distance de Levenshtein peut etre consideree comme une generalisation de la distance de Hamming______________________________________________________________________________________